In [1]:
import torch  
print("torch version:", torch.__version__)  
print("cuda available:", torch.cuda.is_available())  
print("cuda version:", torch.version.cuda)  
if torch.cuda.is_available():  
    print("GPU:", torch.cuda.get_device_name(0))  


torch version: 2.5.1+cu121
cuda available: True
cuda version: 12.1
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU


In [2]:
import torch  
print(torch.__version__)  
print(torch.version.cuda)  


2.5.1+cu121
12.1


In [19]:
import shutil

player_folders = [name for name in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, name))]
print(player_folders)
print("Number of player folders:", len(player_folders))
for folder in player_folders:
    folder_path = os.path.join(root_dir, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
print("All contents deleted from player folders.")

['ibrahimovic', 'etoo', 'depaul', 'nunomendes', 'kaka', 'pedri', 'berkhamp', 'cubarsi', 'pogba', 'ronaldo', 'barella', 'salah', 'neymar', 'cruyff', 'mane', 'raphina', 'mbappe', 'cucurella', 'drogba', 'gullit', 'pele', 'yamal', 'messi', 'dumfries', 'puskas']
Number of player folders: 25
All contents deleted from player folders.


In [21]:
import os
import cv2
from icrawler.builtin import GoogleImageCrawler
from icrawler.downloader import ImageDownloader

# your player list
player_folders = [
    'ibrahimovic', 'etoo', 'depaul', 'nunomendes', 'kaka', 'pedri', 'berkhamp',
    'cubarsi', 'pogba', 'ronaldo', 'barella', 'salah', 'neymar', 'cruyff', 'mane',
    'raphina', 'mbappe', 'cucurella', 'drogba', 'gullit', 'pele', 'yamal',
    'messi', 'dumfries', 'puskas'
]

root_dir = './raw_images'
os.makedirs(root_dir, exist_ok=True)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# custom face-only downloader
class FaceFilterDownloader(ImageDownloader):
    def download(self, task, default_ext, timeout=5, **kwargs):
        file_path = super().download(task, default_ext, timeout, **kwargs)
        if not file_path:
            return None
        try:
            img = cv2.imread(file_path)
            if img is None:
                os.remove(file_path)
                return None
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)
            if len(faces) == 0:
                os.remove(file_path)
                return None
        except Exception:
            os.remove(file_path)
            return None
        return file_path

# scrape each player
for folder in player_folders:
    folder_path = os.path.join(root_dir, folder)
    os.makedirs(folder_path, exist_ok=True)

    existing = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    if existing >= 100:
        print(f"[SKIP] Already has 100 images: {folder}")
        continue

    # fix ambiguous names
    search_map = {
        'depaul': 'Rodrigo De Paul Argentina footballer',
        'berkhamp': 'Dennis Bergkamp footballer',
        'cubarsi': 'Pau Cubarsí Barcelona footballer',
        'nunomendes': 'Nuno Mendes PSG footballer',
        'raphina': 'Raphinha Barcelona footballer',
        'dumfries': 'Denzel Dumfries Netherlands footballer',
        'puskas': 'Ferenc Puskás Hungary footballer',
        'etoo': 'Samuel Eto\'o Cameroon footballer',
        'cruyff': 'Johan Cruyff Netherlands footballer',
        'gullit': 'Ruud Gullit footballer',
        'pele': 'Pelé Brazil footballer',
        'yamal': 'Lamine Yamal Barcelona footballer',
        'cucurella': 'Marc Cucurella Chelsea footballer'
    }
    search_name = search_map.get(folder.lower(), f"{folder} footballer")

    print(f"[SCRAPING] {search_name}...")

    crawler = GoogleImageCrawler(
        downloader_cls=FaceFilterDownloader,
        storage={'root_dir': folder_path}
    )
    crawler.crawl(
        keyword=search_name,
        max_num=150,
        file_idx_offset=0
    )

    valid_imgs = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if len(valid_imgs) > 100:
        for f in valid_imgs[100:]:
            os.remove(os.path.join(folder_path, f))

    print(f"[DONE] {folder} → {min(len(valid_imgs), 100)} face images")

print("✅ all 25 players scraped and filtered")


2025-07-08 17:45:07,422 - INFO - icrawler.crawler - start crawling...
2025-07-08 17:45:07,423 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-07-08 17:45:07,424 - INFO - feeder - thread feeder-001 exit
2025-07-08 17:45:07,424 - INFO - icrawler.crawler - starting 1 parser threads...
2025-07-08 17:45:07,427 - INFO - icrawler.crawler - starting 1 downloader threads...


[SCRAPING] ibrahimovic footballer...


2025-07-08 17:45:08,477 - INFO - parser - parsing result page https://www.google.com/search?q=ibrahimovic+footballer&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 17:45:09,189 - INFO - downloader - image #1	https://upload.wikimedia.org/wikipedia/commons/0/09/Zlatan_Ibrahimovi%C4%87_June_2018.jpg
2025-07-08 17:45:09,269 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Zlatan_Ibrahimovi%C4%87_%2848116890751%29.jpg
2025-07-08 17:45:10,143 - INFO - downloader - image #2	https://cdnuploads.aa.com.tr/uploads/Contents/2019/12/27/thumbs_b_c_b500ef8aa126bc57271e419c59402d1c.jpg
2025-07-08 17:45:10,339 - INFO - downloader - image #3	https://e2.365dm.com/17/08/800x600/skysports-zlatan-ibrahimovic-manchester-united-premier-league-football_4081683.jpg
2025-07-08 17:45:10,469 - ERROR - downloader - Response status code 401, file https://i.guim.co.uk/img/media/f6dbb2a099c5400842157a4d54ea80e1ba871c13/0_76_2760_1656/master/2760.jpg
2025-07-08 17:

[DONE] ibrahimovic → 52 face images
[SCRAPING] Samuel Eto'o Cameroon footballer...


2025-07-08 17:46:17,899 - INFO - parser - parsing result page https://www.google.com/search?q=Samuel+Eto%27o+Cameroon+footballer&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 17:46:18,256 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Samuel_Eto%27o.jpg
2025-07-08 17:46:19,218 - INFO - downloader - image #1	https://ichef.bbci.co.uk/ace/standard/624/cpsprodpb/1C1B/production/_122059170_794d1fe0-9e63-4b42-8a95-0b93ba5b29e3.jpg
2025-07-08 17:46:20,896 - INFO - downloader - image #2	https://images.mlssoccer.com/image/private/t_editorial_landscape_12_desktop/prd-league/p2yo2bsf4ah5x8gqmyrt.jpg
2025-07-08 17:46:21,360 - ERROR - downloader - Response status code 400, file https://media.gettyimages.com/id/651675988/photo/samuel-etoo-cameroon.jpg
2025-07-08 17:46:22,564 - INFO - downloader - image #3	https://cdn.vox-cdn.com/thumbor/83N548Y7XW1CGwcKwbZluNulsr8=/0x0:599x399/1200x800/filters:focal(0x0:599x399)/cdn.vox-cdn.com/uploads/chorus

...

[DONE] messi → 61 face images
[SCRAPING] Denzel Dumfries Netherlands footballer...


2025-07-08 18:10:59,259 - INFO - parser - parsing result page https://www.google.com/search?q=Denzel+Dumfries+Netherlands+footballer&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:10:59,979 - INFO - downloader - image #1	https://static.independent.co.uk/2021/06/13/21/newFile-5.jpg
2025-07-08 18:11:00,300 - INFO - downloader - image #2	https://upload.wikimedia.org/wikipedia/commons/b/b7/Edison_ndreca_inter_egnati_%28cropped_Denzel_Dumfries%29.jpg
2025-07-08 18:11:00,991 - INFO - downloader - image #3	https://ichef.bbci.co.uk/news/1024/branded_sport/147BB/production/_118999838_whatsubject.jpg
2025-07-08 18:11:01,334 - INFO - downloader - image #4	https://cdn.mos.cms.futurecdn.net/U2XBRnd5WHyLd33hB8sQq6-1200-80.jpg
2025-07-08 18:11:01,465 - ERROR - downloader - Response status code 401, file https://i.guim.co.uk/img/media/eb8d3ca3731c9fee8a090baaf5d2ffec6f3da60d/245_54_1802_1081/master/1802.jpg
2025-07-08 18:11:06,660 - INFO - downloader - image #5	https://icdn.sempreinter.com/wp-content/uploa

[DONE] dumfries → 42 face images
[SCRAPING] Ferenc Puskás Hungary footballer...


2025-07-08 18:12:28,396 - INFO - parser - parsing result page https://www.google.com/search?q=Ferenc+Pusk%C3%A1s+Hungary+footballer&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:12:29,434 - INFO - downloader - image #1	https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Puskas_%281971%29.tif/lossy-page1-1200px-Puskas_%281971%29.tif.jpg
2025-07-08 18:12:29,562 - ERROR - downloader - Response status code 401, file https://i.guim.co.uk/img/media/c195bf06cd9859e284ac6ebd0b3f49e7e6b18f79/0_0_1763_2048/master/1763.jpg
2025-07-08 18:12:29,941 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Puskas_Hidegkuti_1954.png
2025-07-08 18:12:30,828 - INFO - downloader - image #2	https://m.media-amazon.com/images/M/MV5BMTI2ODIwODA0NF5BMl5BanBnXkFtZTcwMDE5ODIwMw@@._V1_FMjpg
2025-07-08 18:12:32,622 - INFO - downloader - image #3	https://cdn.britannica.com/32/171432-050-232FFA94/Ferenc-Puskas-1971.jpg
2025-07-08 18:12:34,058 - INFO - downloa

[DONE] puskas → 53 face images
✅ all 25 players scraped and filtered


In [23]:
for folder in player_folders:
    folder_path = os.path.join(root_dir, folder)
    os.makedirs(folder_path, exist_ok=True)

    existing = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    if existing >= 120:  # max total images
        print(f"[SKIP] Already has {existing} images: {folder}")
        continue

    images_to_scrape = 120 - existing

    search_map = {
        'depaul': 'Rodrigo De Paul face',
        'berkhamp': 'Dennis Bergkamp face',
        'cubarsi': 'Pau Cubarsí face',
        'nunomendes': 'Nuno Mendes face',
        'raphina': 'Raphinha face',
        'dumfries': 'Denzel Dumfries face',
        'puskas': 'Ferenc Puskás face',
        'etoo': 'Samuel Eto\'o face',
        'cruyff': 'Johan Cruyff face',
        'gullit': 'Ruud Gullit face',
        'pele': 'Pelé face',
        'yamal': 'Lamine Yamal face',
        'cucurella': 'Marc Cucurella face'
    }
    search_name = search_map.get(folder.lower(), f"{folder} face")

    print(f"[SCRAPING] {search_name} for {images_to_scrape} images...")

    crawler = GoogleImageCrawler(
        downloader_cls=FaceFilterDownloader,
        storage={'root_dir': folder_path}
    )
    crawler.crawl(
        keyword=search_name,
        max_num=images_to_scrape,
        file_idx_offset=existing
    )

    new_total = len([f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
    print(f"[DONE] {folder} → {new_total} images total now")


2025-07-08 18:19:11,347 - INFO - icrawler.crawler - start crawling...
2025-07-08 18:19:11,347 - INFO - icrawler.crawler - starting 1 feeder threads...
2025-07-08 18:19:11,348 - INFO - feeder - thread feeder-001 exit
2025-07-08 18:19:11,348 - INFO - icrawler.crawler - starting 1 parser threads...
2025-07-08 18:19:11,352 - INFO - icrawler.crawler - starting 1 downloader threads...


[SCRAPING] ibrahimovic face for 16 images...


2025-07-08 18:19:12,409 - INFO - parser - parsing result page https://www.google.com/search?q=ibrahimovic+face&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:19:12,519 - INFO - downloader - skip downloading file 000105.jpg
2025-07-08 18:19:12,866 - INFO - downloader - image #2	https://b.fssta.com/uploads/application/soccer/headshots/4046.png
2025-07-08 18:19:12,939 - INFO - downloader - image #3	https://www.celebrity-cutouts.com/wp-content/uploads/2018/05/zlatan-ibrahimovic-celebrity-mask.png
2025-07-08 18:19:13,024 - INFO - downloader - image #4	https://img.a.transfermarkt.technology/portrait/big/3455-1579506060.jpg
2025-07-08 18:19:13,082 - INFO - downloader - image #5	https://images.mlssoccer.com/image/private/t_landscape_tablet/mls-dcu-prd/bd4lffk0eyh83r6awdmz.jpg
2025-07-08 18:19:13,125 - INFO - downloader - image #6	https://b.fssta.com/uploads/application/soccer/headshots/4046.vresize.350.350.medium.75.png
2025-07-08 18:19:16,770 - INFO - downloader - image #7	https://static.toiimg.co

[DONE] ibrahimovic → 119 images total now
[SCRAPING] Samuel Eto'o face for 72 images...


2025-07-08 18:19:22,559 - INFO - parser - parsing result page https://www.google.com/search?q=Samuel+Eto%27o+face&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:19:22,999 - ERROR - downloader - Response status code 401, file https://i.guim.co.uk/img/static/sys-images/Football/Pix/pictures/2010/6/7/1275933449177/Samuel-Etoo-006.jpg
2025-07-08 18:19:23,965 - INFO - downloader - image #1	https://upload.wikimedia.org/wikipedia/commons/f/fb/Samuel_2011.jpg
2025-07-08 18:19:25,531 - INFO - downloader - image #2	https://static.wikia.nocookie.net/the-football-database/images/5/54/Everton_S._Eto%C2%B4o_001.png
2025-07-08 18:19:25,613 - INFO - downloader - image #3	https://img.a.transfermarkt.technology/portrait/big/4257-1683640611.jpg
2025-07-08 18:19:25,839 - INFO - downloader - image #4	https://c8.alamy.com/comp/GBM97T/soccer-uefa-champions-league-semi-final-first-leg-barcelona-v-chelsea-GBM97T.jpg
2025-07-08 18:19:27,581 - INFO - downloader - image #5	https://sortitoutsidospaces.b-cdn.net/media/J

...

[DONE] messi → 62 images total now
[SCRAPING] Denzel Dumfries face for 78 images...


2025-07-08 18:43:07,687 - INFO - parser - parsing result page https://www.google.com/search?q=Denzel+Dumfries+face&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:43:08,153 - ERROR - downloader - Response status code 403, file https://external-preview.redd.it/guarro-denzel-dumfries-has-not-travelled-with-the-squad-to-v0-AvAnosHVLw43svgi3G6ZIcRQGNGZ1Gyzw9M3r2Dnw0k.jpg
2025-07-08 18:43:08,232 - INFO - downloader - image #1	https://sortitoutsi.b-cdn.net/uploads/extractedfiles/bf7RPIMhEM4hCR97auZurOUbk2ymIQCV/37052843.png
2025-07-08 18:43:08,977 - INFO - downloader - image #2	https://img.a.transfermarkt.technology/portrait/big/321528-1746042223.jpg
2025-07-08 18:43:10,664 - INFO - downloader - image #3	https://icdn.sempreinter.com/wp-content/uploads/2022/03/Denzel-Dumfries-10-1024x683.jpg
2025-07-08 18:43:12,111 - INFO - downloader - image #4	https://sortitoutsidospaces.b-cdn.net/images/2v4Q5v79fHbkpIzeg0fDV246klPwV1ek.png
2025-07-08 18:43:13,612 - INFO - downloader - image #5	https://www.fifaro

[DONE] dumfries → 92 images total now
[SCRAPING] Ferenc Puskás face for 67 images...


2025-07-08 18:44:27,827 - INFO - parser - parsing result page https://www.google.com/search?q=Ferenc+Pusk%C3%A1s+face&ijn=0&start=0&tbs=&tbm=isch
2025-07-08 18:44:29,191 - INFO - downloader - image #1	https://www.dx84tech.com/wp-content/uploads/2021/06/Ferenc-PUSKAS-512.png
2025-07-08 18:44:29,891 - INFO - downloader - image #2	http://www.puskas.com/images/news/395/puskas-magyar-fotoarchivum-cut_y9mq.jpg
2025-07-08 18:44:30,478 - ERROR - downloader - Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Ferenc_Pusk%C3%A1s.jpg
2025-07-08 18:44:30,951 - INFO - downloader - image #3	https://i.pinimg.com/736x/a5/d5/d9/a5d5d91412c78b545f72906e72f1d9bb.jpg
2025-07-08 18:44:31,361 - INFO - downloader - image #4	https://c8.alamy.com/comp/A197PP/ferenc-puskas-hungarian-footballer-1927-to-2006-A197PP.jpg
2025-07-08 18:44:31,495 - ERROR - downloader - Response status code 401, file https://i.guim.co.uk/img/media/c195bf06cd9859e284ac6ebd0b3f49e7e6b18f79/0_0_1763_

[DONE] puskas → 109 images total now


## Mediapipe Pipeline for just cropping the raw images and creating eye dataset

In [ ]:
import os
import cv2
import mediapipe as mp
import numpy as np
from tqdm import tqdm  # progress bar

mp_face_mesh = mp.solutions.face_mesh

LEFT_EYE_IDX = [33, 133, 160, 159, 158, 157, 173, 246]
RIGHT_EYE_IDX = [362, 263, 387, 386, 385, 384, 398, 466]

def crop_eyes_balanced(image, landmarks, scale=1.5):
    h, w, _ = image.shape
    eye_points = []
    for idx in LEFT_EYE_IDX + RIGHT_EYE_IDX:
        x = int(landmarks.landmark[idx].x * w)
        y = int(landmarks.landmark[idx].y * h)
        eye_points.append((x, y))
    eye_points = np.array(eye_points)

    x_min, y_min = np.min(eye_points, axis=0)
    x_max, y_max = np.max(eye_points, axis=0)

    center_x = (x_min + x_max) // 2
    center_y = (y_min + y_max) // 2

    width = x_max - x_min
    height = y_max - y_min

    box_width = int(max(width, height * 2) * scale)
    box_height = int(box_width / 2)

    x1 = center_x - box_width // 2
    y1 = center_y - box_height // 2
    x2 = x1 + box_width
    y2 = y1 + box_height

    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(w, x2)
    y2 = min(h, y2)

    return image[y1:y2, x1:x2]

def process_folder(input_dir, output_dir):
    mp_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)
    os.makedirs(output_dir, exist_ok=True)
    
    images = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    count = 0
    
    for img_name in tqdm(images, desc=f"Processing {os.path.basename(input_dir)}"):
        img_path = os.path.join(input_dir, img_name)
        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = mp_mesh.process(img_rgb)
            if not results.multi_face_landmarks:
                continue

            landmarks = results.multi_face_landmarks[0]
            eyes_crop = crop_eyes_balanced(img, landmarks, scale=1.5)
            if eyes_crop.size == 0:
                continue

            save_path = os.path.join(output_dir, f"{count}.jpg")
            cv2.imwrite(save_path, eyes_crop)
            count += 1
        except Exception as e:
            print(f"Skipping {img_name}: {e}")
            continue

    mp_mesh.close()
    print(f"Processed {count} images from {input_dir} → {output_dir}")

# players list
players = [
    'ibrahimovic', 'etoo', 'depaul', 'nunomendes', 'kaka', 'pedri', 'berkhamp',
    'cubarsi', 'pogba', 'ronaldo', 'barella', 'salah', 'neymar', 'cruyff', 'mane',
    'raphina', 'mbappe', 'cucurella', 'drogba', 'gullit', 'pele', 'yamal',
    'messi', 'dumfries', 'puskas'
]

raw_base = './raw_images'
eye_base = './eye_dataset/raw_crops'

for player in players:
    input_dir = os.path.join(raw_base, player)
    output_dir = os.path.join(eye_base, player)
    process_folder(input_dir, output_dir)


I0000 00:00:1751985171.841120   43461 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751985171.842589   84026 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.4), renderer: AMD Radeon 680M (radeonsi, rembrandt, LLVM 20.1.6, DRM 3.63, 6.15.4-200.fc42.x86_64)
W0000 00:00:1751985171.844471   84012 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751985171.848128   84021 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing ibrahimovic: 100%|██████████| 120/120 [00:01<00:00, 94.19it/s] 
I0000 00:00:1751985173.136414   43461 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751985173.137926   84054 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.4), renderer: AMD Radeon 680M (radeonsi, rembran

Processed 83 images from ./raw_images/ibrahimovic → ./eye_dataset/raw_crops/ibrahimovic


W0000 00:00:1751985173.139900   84039 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing etoo:   0%|          | 0/98 [00:00<?, ?it/s]W0000 00:00:1751985173.144911   84043 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
libpng warning: iCCP: known incorrect sRGB profile
Processing etoo: 100%|██████████| 98/98 [00:01<00:00, 87.03it/s]
I0000 00:00:1751985174.274583   43461 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751985174.276326   84085 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.4), renderer: AMD Radeon 680M (radeonsi, rembrandt, LLVM 20.1.6, DRM 3.63, 6.15.4-200.fc42.x86_64)


Processed 76 images from ./raw_images/etoo → ./eye_dataset/raw_crops/etoo


Processing depaul:   0%|          | 0/100 [00:00<?, ?it/s]W0000 00:00:1751985174.278133   84073 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751985174.282069   84071 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing depaul:  30%|███       | 30/100 [00:00<00:01, 58.79it/s]libpng warning: iCCP: known incorrect sRGB profile
[ERROR:0@4475.198] global loadsave.cpp:507 imread_ imread_('./raw_images/depaul/000056.jpg'): can't read data: OpenCV(4.11.0) /io/opencv/modules/imgcodecs/src/grfmt_jpeg2000_openjpeg.cpp:645: error: (-2:Unspecified error) in function 'virtual bool cv::detail::Jpeg2KOpjDecoderBase::readData(cv::Mat&)'
> OpenJPEG2000: tiles are not supported (expected: '(int)comp.dx == 1'), where
>     '(int)comp.dx' is 2
> must be equal to
>     '1' is 1

Processing depaul:

...

Processed 35 images from ./raw_images/messi → ./eye_dataset/raw_crops/messi


Processing dumfries:   0%|          | 0/92 [00:00<?, ?it/s]W0000 00:00:1751985207.292485   85014 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1751985207.298438   85012 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing dumfries: 100%|██████████| 92/92 [00:01<00:00, 56.74it/s]
I0000 00:00:1751985208.922554   43461 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1751985208.924148   85059 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.4), renderer: AMD Radeon 680M (radeonsi, rembrandt, LLVM 20.1.6, DRM 3.63, 6.15.4-200.fc42.x86_64)


Processed 50 images from ./raw_images/dumfries → ./eye_dataset/raw_crops/dumfries


W0000 00:00:1751985208.925810   85040 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
Processing puskas: 100%|██████████| 109/109 [00:00<00:00, 117.25it/s]

Processed 60 images from ./raw_images/puskas → ./eye_dataset/raw_crops/puskas


### Data split into Train Test Val

In [48]:
import os
import shutil
import random

def split_data(source_dir, dest_base_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"

    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        imgs = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(imgs)

        n = len(imgs)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)

        train_imgs = imgs[:n_train]
        val_imgs = imgs[n_train:n_train+n_val]
        test_imgs = imgs[n_train+n_val:]

        for split_name, split_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_dir = os.path.join(dest_base_dir, split_name, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img_file in split_imgs:
                src = os.path.join(cls_path, img_file)
                dst = os.path.join(split_dir, img_file)
                shutil.copy2(src, dst)
    
    print("Dataset split into train, val, test folders.")

# usage example:
raw_crops_dir = './eye_dataset/raw_crops'
final_dataset_dir = './eye_dataset/final'

split_data(raw_crops_dir, final_dataset_dir)


Dataset split into train, val, test folders.


## Applying Augmentation on Training data

In [49]:
from albumentations import (
    Compose, RandomBrightnessContrast, ShiftScaleRotate, HorizontalFlip
)
from albumentations.pytorch import ToTensorV2
import cv2
from torch.utils.data import Dataset, DataLoader
import os

# train augmentations
train_transform = Compose([
    RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.7),
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.7),
    HorizontalFlip(p=0.5),
    ToTensorV2()
])

# val/test no augmentation, just tensor conversion
val_test_transform = Compose([
    ToTensorV2()
])

class EyeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_map = sorted(os.listdir(root_dir))  # ordered classes

        for label_idx, class_name in enumerate(self.class_map):
            class_dir = os.path.join(root_dir, class_name)
            imgs = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            for img in imgs:
                self.images.append(os.path.join(class_dir, img))
                self.labels.append(label_idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']

        return image, label


# Usage example:
train_dataset = EyeDataset('./eye_dataset/final/train', transform=train_transform)
val_dataset = EyeDataset('./eye_dataset/final/val', transform=val_test_transform)
test_dataset = EyeDataset('./eye_dataset/final/test', transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)


In [1]:
pip install torchcam

Note: you may need to restart the kernel to use updated packages.
